In [1]:
pip install telebot SpeechRecognition ffmpeg-python

In [ ]:
import os
import telebot
import speech_recognition
import ffmpeg
import random as rd
from telebot import types
import logging


token = "Ваш токен"

bot = telebot.TeleBot(token, threaded=False)

base = {"user_id": [0, 0]}
phrases = ["Ничего не понятно, но очень интересно.", "Я пока не понимаю, что с этим делать, но обязательно подумаю над этим завтра.",
                        "Кажется ошиблись чатом, бот не знает как быть.", "Всегда готовы выслушать, даже если и не понимаем суть.",
                        "Бот не поймет, это точно ему адресовано?", "Как же я люблю загадки и ребусы.", "Ооо, что-то новенькое и интересное.",
                        "Кажется промахнулся, но где?"]
def oga2wav(filename):
    # Конвертация формата файлов
    new_filename = filename.replace('.oga', '.wav')
    ffmpeg.input(filename).output(new_filename).run()
    return new_filename

def recognize_speech(filename):
    # Перевод голоса в текст + удаление использованных файлов
    text =""
    wav_filename = oga2wav(filename)
    recognizer = speech_recognition.Recognizer()
    with speech_recognition.AudioFile(wav_filename) as source:
        wav_audio=recognizer.record(source)
    try:
        text=recognizer.recognize_google(wav_audio, language='ru')
    except Exception as e:
        logs(e)
        text = f'<i>Бот не смог перевести в текст голос, повторите запрос.</i>'
    finally:
        if os.path.exists(filename):
            os.remove(filename)
        if os.path.exists(wav_filename):
            os.remove(wav_filename)
        return text

def download_file(bot, file_id):
    # Скачивание файла от пользователя
    file_info = bot.get_file(file_id)
    downloaded_file = bot.download_file(file_info.file_path)
    filename = file_id+file_info.file_path
    filename = filename.replace('/', '_')
    with open(filename,'wb') as f:
        f.write(downloaded_file)
    return filename

@bot.message_handler(commands=['start'])
def say_hi(message):
    # Отправляет персонализированный привет пользователю.
    sending_message(message.chat.id, f'<i>Приветствую, <b>{message.from_user.first_name}</b>! Больше о боте - /help.</i>',  'html', '')
    stopping_game(message)

def sending_message(id, text, mode, markup):
    try:
       bot.send_message(id, text, parse_mode=mode, reply_markup=markup)
    except Exception as e:
        logs(e)
        pass

def logs(e):
    logging.basicConfig(format='%(asctime)s %(message)s', level=logging.ERROR, filename ="py_log.log", filemode="w")
    logging.error(e)


@bot.message_handler(content_types=['voice'])
def transcript(message):
    # Отправляет текст в ответ на голосовое сообщение
    filename1 = download_file(bot, message.voice.file_id)
    text = recognize_speech(filename1)
    sending_message(message.chat.id, f'<i>{text}</i>',  'html', '')
    stopping_game(message)


@bot.message_handler(content_types=['sticker'])
    # Отправляет стикер в ответ на стикер пользователя
def send_sticker(message):
    sti = open(str(rd.randint(1,5)) + ".tgs", "rb")
    try:
        bot.send_sticker(message.chat.id, sti)
    except Exception as e:
        logs(e)
        pass
    stopping_game(message)

@bot.message_handler(commands=['play_game'])
def play_game(message):
    # Запускает угадайку по команде, если игра не была запущена ранее, предлагает выход из угадайки
    markup=types.InlineKeyboardMarkup()
    bot1 = types.InlineKeyboardButton('выход из игры', callback_data='exit'+ '_' + str(message.from_user.id))
    markup.add(bot1)
    if base.setdefault(message.from_user.id) != None:
        sending_message(message.chat.id, f'<i>Вы уже в игре.</i>', 'html', markup)
    else:
        sending_message(message.chat.id, f'<i>Игра угадайка. Бот загадал число от 1 до 10, угадайте какое, на игру дается 5 попыток.</i>',
                         'html', markup)
        base[message.from_user.id] = []
        base[message.from_user.id].append(rd.randint(1, 10))
        base[message.from_user.id].append(5)


@bot.callback_query_handler(func=lambda call: True)
def callback_query(call):
    # Обрабатывает ответные сообщение из доп.меню
    req = call.data.split('_')
    try:
        if req[0] == 'exit':
            bot.send_message(call.message.chat.id, f'<i>Вы вышли из игры.</i>', parse_mode='html')
            base.pop(int(req[1]))
            bot.delete_message(call.message.chat.id, call.message.message_id)
        elif req[0] == 'show':
            mp4 = open("bot.mp4", "rb")
            bot.send_video(call.message.chat.id, mp4)
            bot.delete_message(call.message.chat.id, call.message.message_id)
    except Exception as e:
        logs(e)
        pass


@bot.message_handler(commands=['help'])
def help(message):
    # Предоставляет больше информации о боте по команде
    sending_message(message.chat.id,
        f'<i>Бот переводит голосовые сообщения в русскоязычный текст, отправляет стикер, играет в угадайку.</i>', 'html', '')
    stopping_game(message)

@bot.message_handler(regexp='\\d{1,}')
def game(message):
    # Обрабатывает числа в угадайке и те, что отправлены вне игры
    if message.text.isdigit() != True:
        send_unknown_message(message)
        stopping_game(message)
    else:
        numb = int(message.text)
        if base.setdefault(message.from_user.id) != None and base[message.from_user.id][1] > 1:
            attempt = base[message.from_user.id][1]
            attempt -= 1
            rdint =  base[message.from_user.id][0]
            base[message.from_user.id][1] -= 1
            if numb > 10 or numb < 1:
                sending_message(message.chat.id, f"<i>Введенное значение <b>{numb}</b> не соответствует числу в диапозоне от 1 до 10.\nИз 5 попыток осталось: <b>{attempt}</b>.</i>",
                                'html', '')
            elif numb < rdint:
                sending_message(message.chat.id, f"<i>Введенное число <b>{numb}</b> меньше загаданного числа.\nИз 5 попыток осталось: <b>{attempt}</b>.</i>", 'html', '')
            elif numb > rdint:
                sending_message(message.chat.id, f"<i>Введенное число <b>{numb}</b> больше загаданного числа.\nИз 5 попыток осталось: <b>{attempt}</b>.</i>",
                            'html', '')
            else:
                markup=types.InlineKeyboardMarkup()
                bot1 = types.InlineKeyboardButton('показать бота', callback_data='show')
                markup.add(bot1)
                sending_message(message.chat.id, f"<i>Вы угадали! Поздравляем!!! Бот скачет от радости!!!</i>", 'html', markup)
                base.pop(message.from_user.id)
        elif base.setdefault(message.from_user.id) != None and base[message.from_user.id][1] == 1:
            sending_message(message.chat.id,   f"<i>Попытки закончились. Игра завершена. Загаданное число: <b>{base[message.from_user.id][0]}</b></i>.", 'html', '')
            base.pop(message.from_user.id)
        else:
            send_unknown_message(message)


def stopping_game(message):
    # Завершает игру, применяется в других функциях, напромер при отправке во время игры голосового сообщения
    if base.setdefault(message.from_user.id) != None:
        base.pop(message.from_user.id)
        sending_message(message.chat.id, f'<i>Игра не была завершена. Бот вышел из игры. Запустите игру снова.</i>',  'html', '')

@bot.message_handler(regexp='\\D')
def another_text(message):
    # Отправляет что не знает как быть с текстовыми сообщениями
    send_unknown_message(message)

@bot.message_handler(func=lambda message: True, content_types=['audio', 'photo', 'video','document', 'location','contact', 'video_note','animation',
                                                              'game', 'venue', 'dice'])
def send_unknown_message(message):
    # Отправляет что не знает как быть по остальным основным типам сообщений
    numb = rd.randint(0, (len(phrases)-1))
    text= phrases[numb]
    try:
        bot.reply_to(message, f'<i>{text}</i>', parse_mode='html')
    except Exception as e:
        logs(e)
        pass
    stopping_game(message)

while True:
    try:
        bot.infinity_polling(timeout=3, long_polling_timeout = 3)
    except Exception as e:
        logs(e)
        pass
